In [ ]:
# Import pre-requesite
import cv2
import matplotlib.pyplot as plt 
import numpy as np
import os
from PIL import Image
import random
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input,Conv2D, MaxPool2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0

In [ ]:
# Get CWD
os.getcwd()

In [ ]:
image_list[0]

In [ ]:
# Get Data
base_path = os.getcwd() + '/Dataset'
image_list = os.listdir('./Dataset')
image0_path = os.path.join(base_path,image_list[0])
image0 = Image.open(image0_path)
plt.imshow(image0)
plt.show()

In [ ]:
def contrast_stretch(im):
    """
    Performs a simple contrast stretch of the given image, from 5-100%.
    """
    in_min = np.percentile(im, 5)
    in_max = np.percentile(im, 100)

    out_min = 0.0
    out_max = 255.0

    out = im - in_min
    out *= ((out_min - out_max) / (in_min - in_max))
    out += in_min

    return out


def get_ndvi(image_path):
    """
    Transform a raw image to ndvi image
    """
    image = cv2.imread(image_path) 
    b, g, r = cv2.split(image)

    bottom = (r.astype(float) + b.astype(float))
    bottom[bottom == 0] = 0.00001  # Make sure we don't divide by zero!
    ndvi_image = (r.astype(float) - b) / bottom
    ndvi_image = contrast_stretch(ndvi_image)
    ndvi_image = ndvi_image.astype(np.uint8)
    return ndvi_image

In [ ]:
ndvi0 = get_ndvi(image0_path)
ndvi0

In [ ]:
ndvi0

In [ ]:
plt.imshow(ndvi0)
plt.show()

#  NDVI / NO2 Dataset

In [ ]:
# Get NDVI images
def ndvi_images():
    ndvi_img_list = []
    for i in range(0,len(image_list)):
        img_path = os.path.join(base_path,image_list[i])
        ndvi_img = get_ndvi(img_path)
        ndvi_img_list.append(ndvi_img)
    ndvi_img_numpy = np.array(ndvi_img_list)
    return ndvi_img_numpy

In [ ]:
# Get NO2 label within [0,2]
def no2_labels():
    label_list = []
    for i in range(0,82):
        random_label = random.randint(0,2)
        label_list.append(random_label)
    label_numpy = np.array(label_list)
    return label_numpy

In [ ]:
# Info on X and Y
print(ndvi_img_numpy.shape)
print(label_numpy.shape)

# Modeling Conv1D

In [ ]:
model = Sequential([
    tf.keras.layers.Conv1D(8,(3),input_shape=(1944,2592)),
    tf.keras.layers.MaxPool1D(2,2),
    tf.keras.layers.Conv1D(16,(3)),
    tf.keras.layers.MaxPool1D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3,activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
x_train = ndvi_img_numpy
y_train = label_numpy

In [ ]:
history = model.fit(x_train,y_train,epochs=10)

In [ ]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["City name", "Area", "Population", "Annual Rainfall"]
x.add_row(["Adelaide", 1295, 1158259, 600.5])
x.add_row(["Brisbane", 5905, 1857594, 1146.4])
x.add_row(["Darwin", 112, 120900, 1714.7])
x.add_row(["Hobart", 1357, 205556, 619.5])
x.add_row(["Sydney", 2058, 4336374, 1214.8])
x.add_row(["Melbourne", 1566, 3806092, 646.9])
x.add_row(["Perth", 5386, 1554769, 869.4])
print(x)

# Modeling Conv2D

In [ ]:
# https://pythonexamples.org/python-opencv-cv2-resize-image/

def scale_down(image):
    src = image
    #percent by which the image is resized
    scale_percent = 25

    #calculate the 50 percent of original dimensions
    width = int(src.shape[1] * scale_percent / 100)
    height = int(src.shape[0] * scale_percent / 100)

    # dsize
    dsize = (width, height)

    # resize image
    output = cv2.resize(src, dsize)
    return output

In [ ]:
ndvi_img_list = []
for i in range(0,len(image_list)):
    img_path = os.path.join(base_path,image_list[i])
    ndvi_img = get_ndvi(img_path)
    ndvi_img = scale_down(ndvi_img)
    ndvi_img_list.append(ndvi_img)
ndvi_img_numpy = np.array(ndvi_img_list)

In [ ]:
x_train = ndvi_img_numpy
y_train = label_numpy
print(x_train.shape)
print(y_train.shape)

In [ ]:
x_train = np.expand_dims(x_train,axis=3)
print(x_train.shape)

In [ ]:
model = Sequential([
    tf.keras.layers.Conv2D(8,(3,3),input_shape=(486,648,1)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Conv2D(16,(3,3)),
    tf.keras.layers.MaxPool2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3,activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(x_train,y_train,epochs=10)

# Modeling MobileNetV2

In [ ]:
ndvi0.shape

In [ ]:
ndvi2rgb = cv2.cvtColor(ndvi0,cv2.COLOR_GRAY2RGB)

In [ ]:
ndvi2rgb.shape

In [ ]:
x_train = ndvi_img_numpy
y_train = label_numpy
print(x_train.shape)
print(y_train.shape)

In [ ]:
xtrain2rgb = cv2.cvtColor(x_train,cv2.COLOR_GRAY2RGB)

In [ ]:
len(ndvi_img_list)

In [ ]:
ndvi_rgb_list = []
for i in range(0,len(ndvi_img_list)):
    ndvi_rgb = cv2.cvtColor(ndvi_img_list[i],cv2.COLOR_GRAY2RGB)
    ndvi_rgb_list.append(ndvi_rgb)
ndvi_rgb_numpy = np.array(ndvi_rgb_list)
print(ndvi_rgb_numpy.shape)

In [ ]:
x_train = ndvi_rgb_numpy
y_train = label_numpy

In [ ]:
mobilenetv2 = MobileNetV2(include_top=False, weights='imagenet',input_shape=(486,648,3))
for layer in mobilenetv2.layers:
        layer.trainable = False
mobilenetv2_preprocess = tf.keras.applications.mobilenet_v2.preprocess_input

In [ ]:
mobilenetv2.summary()

In [ ]:
# Transfer Learning
input_shape = (486,648,3)
img_in = Input(shape=input_shape, name='img_in')
x = mobilenetv2_preprocess(img_in)
x = mobilenetv2(img_in, training=True)
x = GlobalAveragePooling2D()(x)
# Classification layer
output = Dense(3, activation='softmax', name='dense')(x)
# Final model
model = Model(inputs=[img_in], outputs=output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(x_train,y_train,epochs=10)

# Modeling EfficientNetB0

In [ ]:
ndvi_rgb_list = []
for i in range(0,len(ndvi_img_list)):
    ndvi_rgb = cv2.cvtColor(ndvi_img_list[i],cv2.COLOR_GRAY2RGB)
    ndvi_rgb_list.append(ndvi_rgb)
ndvi_rgb_numpy = np.array(ndvi_rgb_list)
print(ndvi_rgb_numpy.shape)

In [ ]:
x_train = ndvi_rgb_numpy
y_train = label_numpy

In [ ]:
efficientnetb0 = EfficientNetB0(include_top=False, weights='imagenet',input_shape=(486,648,3))
for layer in efficientnetb0.layers:
        layer.trainable = False
efficientnetb0_preprocess = tf.keras.applications.efficientnet.preprocess_input

In [ ]:
efficientnetb0.summary()

In [ ]:
# Transfer Learning
input_shape = (486,648,3)
img_in = Input(shape=input_shape, name='img_in')
x = efficientnetb0_preprocess(img_in)
x = efficientnetb0(img_in, training=True)
x = GlobalAveragePooling2D()(x)
# Classification layer
output = Dense(3, activation='softmax', name='dense')(x)
# Final model
model = Model(inputs=[img_in], outputs=output)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [ ]:
history = model.fit(x_train,y_train,epochs=10)

In [ ]:
# Prediction
model.predict(x_train)

In [ ]:
model.predict([x_train[0]])

In [ ]:
x_train.shape

In [ ]:
# Inference time for EfficientNetB0

In [153]:
np.expand_dims(x_train[0],axis=0).shape

(1, 1944, 2592)

In [152]:
tests = []
for i in range(0,82):
    test = np.expand_dims(x_train[i],axis=0)
    tests.append(test)

In [ ]:
from tqdm import tqdm
for i in tqdm(range(0,82)):
    model.predict(tests[i])

In [ ]:
import tensorflow.keras.backend as K

trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total params: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable params: {:,}'.format(trainable_count))
print('Non-trainable params: {:,}'.format(non_trainable_count))

In [ ]:
def get_model_params(model):
    trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
    non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

    print('Total params: {:,}'.format(trainable_count + non_trainable_count))
    print('Trainable params: {:,}'.format(trainable_count))
    print('Non-trainable params: {:,}'.format(non_trainable_count))
    return (trainable_count + non_trainable_count),trainable_count,non_trainable_count

In [ ]:
get_model_params(model)[2]

In [ ]:
def save_model(model_name,model):
    model.save(model_name+".h5")
    print(f"Model saved as {model_name}.h5")

In [ ]:
save_model("conv1D",conv1D_model)

In [ ]:
def load_model(model_name):
    model = tf.keras.models.load_model(model_name)
    return model

In [ ]:
conv1D = load_model("conv1D.h5")
conv1D.summary()
conv1D.weights

In [ ]:
len(conv1D.predict(x_train))

In [ ]:
conv1D_model = Sequential([
    tf.keras.layers.Conv1D(8,(3),input_shape=(1944,2592)),
    tf.keras.layers.MaxPool1D(2,2),
    tf.keras.layers.Conv1D(16,(3)),
    tf.keras.layers.MaxPool1D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(3,activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

In [158]:
def ndvi_small_image():
    ndvi_img_list = []
    for i in range(0,len(image_list)):
        img_path = os.path.join(base_path,image_list[i])
        ndvi_img = get_ndvi(img_path)
        ndvi_img = scale_down(ndvi_img)
        ndvi_img_list.append(ndvi_img)
    return ndvi_img_list

In [159]:
x_train_2D = np.array(ndvi_small_image())
x_train_2D = np.expand_dims(x_train_2D,axis=3)

In [163]:
x_train_2D[0].shape

(486, 648, 1)

In [164]:
x_train = []
for i in range(0,82):
    x_train_i = np.expand_dims(x_train_2D[i],axis=0)
    x_train.append(x_train_i)

In [167]:
x_train[0].shape

(1, 486, 648, 1)